In [1]:
# import the required libraries 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
from google.oauth2.credentials import Credentials
import pickle 
import os.path 
import base64 
import email 
import re
import datetime
import pandas as pd
from tqdm import tqdm
from dateutil import parser
from bs4 import BeautifulSoup
import logging
import requests
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)

In [3]:
def buscar_key(diccionario, key_buscada):
    if key_buscada in diccionario:
        return diccionario[key_buscada]
    for key, value in diccionario.items():
        if isinstance(value, dict):
            result = buscar_key(value, key_buscada)
            if result is not None:
                return result
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    result = buscar_key(item, key_buscada)
                    if result is not None:
                        return result
    return None


def convert_to_float(amount):
    # Reemplazar comas por puntos
    amount = amount.replace(',', '.')
    # Encontrar la última aparición de un punto
    last_dot_index = amount.rfind('.')
    if last_dot_index != -1:
        # Eliminar todos los puntos anteriores al último
        amount = amount[:last_dot_index].replace('.', '') + amount[last_dot_index:]
    return float(amount)

In [4]:
SCOPES = ["https://www.googleapis.com/auth/userinfo.profile  https://www.googleapis.com/auth/gmail.readonly"]

# Cargar credenciales
def load_credentials():
    creds = None
    if os.path.exists('utils/token.json'):
        creds = Credentials.from_authorized_user_file('utils/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('utils/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('utils/token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

creds = load_credentials()

In [5]:
userinfo_endpoint = 'https://www.googleapis.com/oauth2/v1/userinfo'
params = {'alt': 'json', 'access_token': creds.token}
response = requests.get(userinfo_endpoint, params=params)
user_info = response.json()

In [6]:
user_info

{'id': '109910674197465709992',
 'name': 'Emiliano Lo Sasso',
 'given_name': 'Emiliano',
 'family_name': 'Lo Sasso',
 'picture': 'https://lh3.googleusercontent.com/a/ACg8ocKFcbHT7qMZmlcTTAJKhW4NmklWZIXmzi_w4_KbOkqAcuUM=s96-c'}

In [7]:
gmail_service = build('gmail', 'v1', credentials=creds)

2024-06-17 12:32:04,464 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


In [8]:
query = f"from: prismahome@prismasystems.com.ar after: {datetime.date.today()}"

# We can also pass maxResults to get any number of emails. Like this: 
result = gmail_service.users().messages().list(userId='me', q = query).execute() 
messages = result.get('messages')


2024-06-17 12:32:07,198 - googleapiclient.http - WARNING - Encountered 403 Forbidden with reason "insufficientPermissions"


HttpError: <HttpError 403 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages?q=from%3A+prismahome%40prismasystems.com.ar+after%3A+2024-06-17&alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">

In [71]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2.service_account import Credentials as ServiceAccountCredentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Alcances
GMAIL_SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
SHEET_SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# ID de la hoja de cálculo
SAMPLE_SPREADSHEET_ID = '15HjxuJARpG_lpj_wbmSRhEWC1aRP2VXy4iyFl4TQuwU'
SERVICE_ACCOUNT_FILE = 'utils/service_account.json'  # Ruta al archivo JSON de la cuenta de servicio
USER_CREDENTIALS_FILE = 'utils/credentials.json'  # Ruta al archivo JSON de credenciales del usuario

def get_gmail_service():
    creds = None
    # El archivo token.json almacena los tokens de acceso y actualización del usuario.
    if os.path.exists('utils/token.json'):
        creds = Credentials.from_authorized_user_file('utils/token.json', GMAIL_SCOPES)
    # Si no hay credenciales válidas disponibles, solicita al usuario que inicie sesión.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(USER_CREDENTIALS_FILE, GMAIL_SCOPES)
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para la próxima ejecución
        with open('utils/token.json', 'w') as token:
            token.write(creds.to_json())

    # Crea el servicio de Gmail
    return build('gmail', 'v1', credentials=creds)

def get_sheets_service():
    # Carga las credenciales de la cuenta de servicio
    creds = ServiceAccountCredentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SHEET_SCOPES)
    # Crea el servicio de Sheets
    return build('sheets', 'v4', credentials=creds)
    

def get_daily_mails():
    # Get gmail service
    
    service = get_gmail_service()
    # Get list of emails
    # Define query
    query = f"from: prismahome@prismasystems.com.ar after: {datetime.date.today()}"

    # We can also pass maxResults to get any number of emails. Like this: 
    result = service.users().messages().list(userId='me', q = query).execute() 
    messages = result.get('messages')

    results = []

    logger.info("Getting today's mails")
    for message in tqdm(messages):
        # Get info from mail
        txt = service.users().messages().get(userId='me', id=message['id']).execute()

        temp = pd.DataFrame(txt['payload']['headers'])
        subject = temp.loc[temp.name == 'Subject', 'value'].values[0]
        
        if 'débito' not in subject or 'debito' not in subject:
            # Get amount spent
            regex = r'\$\s*([0-9]+(?:[.,][0-9]{3})*(?:[.,][0-9]{2})?)'

            texto = txt['snippet']
            amount = re.findall(regex, texto)
            amount = convert_to_float(amount[0].split('$')[-1].strip())


            # Get date
            purchase_date = temp.loc[temp.name == 'Date','value'].values[0]
            purchase_date = parser.parse(purchase_date).date()

            # Save results
            results.append((purchase_date, amount))
        else:
            pass
    
    logger.info(f"Finished! {len(results)} mails found!")

    return results


In [82]:
results  

[(datetime.date(2024, 6, 16), 15500.0),
 (datetime.date(2024, 6, 16), 48350.0),
 (datetime.date(2024, 6, 16), 19800.0)]

In [72]:
results = get_daily_mails()

2024-06-16 19:46:12,384 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0
2024-06-16 19:46:13,053 - root - INFO - Getting today's mails
100%|██████████| 3/3 [00:00<00:00,  3.24it/s]
2024-06-16 19:46:13,982 - root - INFO - Finished! 3 mails found!


In [76]:
# Obtener el servicio de Sheets autenticado por la cuenta de servicio
sheets_service = get_sheets_service()

2024-06-17 10:20:36,716 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


In [81]:
try:
    # Datos a agregar
    values = [
        ["tuti", 200, str(datetime.date.today())]
    ]

    # Configurar el cuerpo de la solicitud
    body = {
        'values': values
    }

    # Llamar a la API de Sheets para agregar los datos
    result = sheets_service.spreadsheets().values().append(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        range="movements",  # El nombre de la hoja donde se agregan los datos
        valueInputOption="USER_ENTERED",  # 'RAW' o 'USER_ENTERED'
        insertDataOption="INSERT_ROWS",  # 'OVERWRITE' o 'INSERT_ROWS'
        body=body
    ).execute()

    logger.info("Table updated")

except HttpError as error:
    print(f"An error occurred: {error}")

2024-06-17 10:25:30,025 - root - INFO - Table updated


In [45]:
if len(results) == 0:
            pass
else:
    # Obtener el servicio de Sheets autenticado por la cuenta de servicio
    sheets_service = get_sheets_service()
    sheet = sheets_service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="movements").execute()

test_results = pd.DataFrame(result['values'])
test_results.columns = test_results.iloc[0,:]
test_results = test_results.iloc[1:,:]
test_results.reset_index(drop = True, inplace = True)
test_results

,user,budget,date
0,emi,100,16/06/2024


In [66]:
service = get_gmail_service()

2024-06-16 19:44:38,420 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


In [67]:
# Get list of emails
# Define query
query = f"from: prismahome@prismasystems.com.ar after: {datetime.date.today()}"

# We can also pass maxResults to get any number of emails. Like this: 
result = service.users().messages().list(maxResults=50, userId='me', q = query).execute() 
messages = result.get('messages')

In [69]:
results = []

for message in tqdm(messages):
    # Get info from mail
    txt = service.users().messages().get(userId='me', id=message['id']).execute()

    temp = pd.DataFrame(txt['payload']['headers'])
    subject = temp.loc[temp.name == 'Subject', 'value'].values[0]
    
    if 'débito' not in subject or 'debito' not in subject:
        # Get amount spent
        regex = r'\$\s*([0-9]+(?:[.,][0-9]{3})*(?:[.,][0-9]{2})?)'

        texto = txt['snippet']
        amount = re.findall(regex, texto)
        amount = convert_to_float(amount[0].split('$')[-1].strip())


        # Get date
        purchase_date = temp.loc[temp.name == 'Date','value'].values[0]
        purchase_date = parser.parse(purchase_date).date()

        # Save results
        results.append((purchase_date, amount))
    else:
        pass

100%|██████████| 3/3 [00:01<00:00,  2.54it/s]
